# Si4713 Functional Test 
- https://www.silabs.com/documents/public/data-sheets/Si4712-13-B30.pdf
- https://github.com/adafruit/Adafruit_CircuitPython_SI4713
- https://cdn-shop.adafruit.com/datasheets/SiLabs+Programming+guide+AN332.pdf

In [1]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [2]:
# paths = [' ']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [3]:
paths =[['..', '..', '..', '..', '..', 'Signal Generators', 'bitbucket', 'github', 'codes'],
        ['..', '..', '..', '..', '..', 'FX2LP', 'bitbucket', 'github', 'codes'], 
        ['..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],
        ['..', '..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [4]:
%pylab inline
 
from array import array
from utilities.adapters import peripherals
from fm_transceivers.si47xx import Si4713
import fx2lp

Populating the interactive namespace from numpy and matplotlib


## Generators

## Debug mode?

In [5]:
cls = Si4713

cls.DEBUG_MODE_SHOW_BUS_DATA = False         # whether to show bus data. 
cls.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## Generators

In [6]:
with_hardware_device = False

if with_hardware_device:
    _i2c = peripherals.I2C.get_Ftdi_i2c()
    pin_reset = peripherals.Pin.get_Ftdi_pin(pin_id = 1)
else:
    _i2c = pin_reset = None  # using None for testing without actual hardware device.

    
bus = peripherals.I2C(_i2c)


****** Virtual device. Data may not be real ! ******



In [7]:
bus = fx2lp.I2C(as_400KHz = True)
pin_reset = fx2lp.GPIO().Pin(id = 1, mode = fx2lp.Pin.OUT, value = 1)

In [8]:
freq = 88.80e6

si = Si4713(bus, pin_reset = pin_reset, freq = freq)
si.start()

## Noise Level Measurement

In [ ]:
plt.figure(figsize = (15, 5))
 
noise_levels = si.scan_noise_levels()
nl = np.array(noise_levels).T
plt.plot(nl[0] / 1e6, nl[1]);

In [ ]:
noise_levels_sorted = sorted(noise_levels, key = lambda x: x[1])
noise_levels_sorted[:30]

## RDS

In [ ]:
si.set_rds(program_id = 0x0520,
           station_name = "Wei Lin",
           radio_text = "My Radio Station !",
           program_type_code = 4,
           repeat_count = 3, message_count = 1, rds_mix_ratio = 50,  
           rds_fifo_size = 20,
           enable = True)

In [ ]:
# while True:
#     si.set_rds_buffer(message = time.strftime("%Y-%m-%d %H:%M:%S"), use_FIFO = True)
#     time.sleep(10)

In [ ]:
si.set_rds(enable = False)

## Member functions test

In [ ]:
# for f in dir(Si4713):
#     if not f.startswith('_'):
#         if f.islower():
#             if callable(getattr(Si4713, f)):
#                 print('si.{}()'.format(f))

In [ ]:
# si.apply_signal()

In [ ]:
si.close()

In [ ]:
si.start()

In [ ]:
si.enable(False)

In [ ]:
si.enable(True)

In [ ]:
si.enable_output(False)

In [ ]:
si.enable_output(True)

In [ ]:
# si.enable_output_channel(0)

In [ ]:
# si.load_registers()

In [ ]:
si.mute(True)

In [ ]:
si.mute(False)

In [ ]:
si.mute_line_input(True)

In [ ]:
si.mute_line_input(False)

In [ ]:
si.stereo = False
print(si.stereo)

In [ ]:
si.stereo = True
print(si.stereo)

In [ ]:
si.pause()

In [ ]:
si.resume()

In [ ]:
si.power_down()

In [ ]:
si.power_up()

In [ ]:
si.init()

In [ ]:
si.power_down()

In [ ]:
si.reset()

In [ ]:
# si.select_freq_source(0)

In [ ]:
# si.select_phase_source(0)

In [ ]:
si.set_frequency(88.9e6)

In [ ]:
si.set_frequency(88.8e6)

In [ ]:
si.set_gpio_direction()

In [ ]:
si.set_gpio_level()

In [ ]:
# si.set_phase(0)

In [ ]:
si.set_power(88)

In [ ]:
si.set_power(115)

In [ ]:
si.stop()

In [ ]:
si.start()

In [ ]:
si.update()

In [ ]:
si.read_all_registers()

In [ ]:
si.write_all_registers()

In [ ]:
si.print()